### boor-system

In [10]:
import re
import sys
sys.path.append('/Users/baza/dev/python-boorstat/')

#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
import plotly.graph_objs as go

import boorstat

### boor-title

Pushkin's Duels CSV

### boor-short-content

[empty]

### boor-full-content

There are more than 25 known Pushkin's duels.<br/>
Let's make CSV containing these duels details.<br/>
Here its full content:

```
year,opponent_name,opponent_descr,cause,pushkin_shot,opponent_shot
1816,Paul Hannibal,uncle,during a ball Paul lugged away Pushkin’s girlfriend miss Loshakova,0,0
1817,Pyotr Kaverin,friend,Kaverin’s facetious poem,0,0
1819,Kondratiy Ryleev,poet,Ryleev told a joke about Pushkin at a high society gathering,0,0
1819,Wilhelm Kiichelbecker,friend,funny verses about Küchelbecker,0,1
1819,Modest Korf,Ministry of justice worker,Pushkin’s drunk manservant pestered Korf’s servant, who finally beat Pushkin’s servant up,0,0
1819,Denisevich,Major,Pushkin behaved provocatively in theater: he yelled at actors,0,0
1820,Orlov Fedor and Alexey Alexeev,,Orlov and Alexeev reprimanded Pushkin for being drunk and trying to play pool,0,0
1821,Deguilly,French military officer,a quarrel under unclear circumstances,0,0
1822,Semyon Starov,lieutenant colonel,a conflict occurred because of a restaurant orchestra at a casino where both indulged in gambling,1,1
1822,Ivan Lanov,65-year-old state councilor,a quarrel during a holiday dinner,0,0
1822,Todor Balsh,Moldavian nobleman,Balsh’s wife Maria responded to Pushkin’s question in an impolite manner,1,1
1822,Skartla Pruncul,Bessarabian landowner,they were seconds at someone else’s duel and could not agree upon the rules of the duel,0,0
1822,Seweryn Potocki,Active Privy Councillor,discussion about serfdom at the dinner table,0,0
1822,Rutkowski,captain,Pushkin did not believe that a hailstone can weigh up to 3 pounds (which is possible) and made fun of the retired captain,0,0
1822,Inglezi,Chisinau tycoon,Pushkin coveted his wife (a gypsy woman Ludmila Shekora),0,0
1832,Alexander Zubov,General Staff warrant officer,Pushkin had caught Zubov on cheating during a game of cards,0,1
1823,Ivan Rousseau,young writer,Pushkin’s personal dislike for this person,0,0
1826,Nikolay Turgenev,one of the leaders of the Union of Welfare and a member of the Northern Society,0,0
1827,Vladimir Solomirskiy,artillery officer,the officer’s female friend Sofia to whom Pushkin was personally attracted,0,0
1828,Alexander Golitsyn,Minister of Education,Pushkin wrote a bold epigram so the Minister arranged a rough interrogation,0,0
1828,Lagrenée,French Embassy Secretary in St.Petersburg,an unknown girl at a ball,0,0
1829,Mr. Hvostov,Foreign Office worker,Hvostov was dissatisfied by Pushkin’s epigrams,0,0
1836,Nikolay Repin,,Repin was dissatisfied with Pushkin’s poems about him,0,0
1836,Semyon Hlustin,Foreign Office worker,Hlustin did not approve of Pushkin’s poetry,0,0
1836,Vladimir Sollogub,minor Russian writer,Sologub’s unflattering remarks about the poet’s wife Natalia.0,0
1836,George d’Anthès,French officer,an anonymous letter which stated that Pushkin’s wife had been cheating on her husband with d’Anthès,0,0
1837,George d’Anthès,French officer,an anonymous letter which stated that Pushkin’s wife had been cheating on her husband with d’Anthès,1,1
```

Here is also direct link to download <a href="https://raw.githubusercontent.com/boorstat/boorstat-files/master/lit/pushkin/duels.csv">Pushkin's duels csv</a>.<br/>

Source for this data:<br/>
https://rinatim.com/2016/12/03/alexander-pushkins-duels/<br/>
http://d-push.net<br/>
https://ru.wikipedia.org/wiki/Пушкин,_Александр_Сергеевич<br/>

And now let's try to use it and plot some visual representation of Pushkin's duels into real shots conversion.<br/>
Starting with imports:

In [12]:
from io import StringIO

import requests
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go

DUELS_CSV_URL = 'https://raw.githubusercontent.com/boorstat/boorstat-files/master/lit/pushkin/duels.csv'

Getting csv as pandas dataframe:

In [17]:
df = pd.DataFrame.from_csv(StringIO(requests.get(DUELS_CSV_URL).text), index_col=None)
df.head()

,year,opponent_name,opponent_descr,cause,pushkin_shot,opponent_shot
0,1816,Paul Hannibal,uncle,during a ball Paul lugged away Pushkin’s girlf...,0,0.0
1,1817,Pyotr Kaverin,friend,Kaverin’s facetious poem,0,0.0
2,1819,Kondratiy Ryleev,poet,Ryleev told a joke about Pushkin at a high soc...,0,0.0
3,1819,Wilhelm Kiichelbecker,frien,funny verses about Küchelbecker,0,1.0
4,1819,Modest Korf,Ministry of justice worker,Pushkin’s drunk manservant pestered Korf’s ser...,0,0.0


This is the most top level of our visualization script:

In [14]:
# roman = idiot.from_json()

# data = parse_parts(roman)
# traces = prepare_data(data)
# plot(traces)

Final plot function:

In [15]:
def plot(traces):
    layout = go.Layout(title='Idiot Characters')
    fig = go.Figure(data=traces, layout=layout)

    return py.iplot(
        fig,
        filename='idiot-characters',
        sharing='public')

Couple of functions where idiot parts and chapters parsed and charactes rates are set:

In [16]:
def rate_characters(chapter):
    characters = {}

    for char, regexps in CHARACTERS.items():
        characters[char] = sum([len(re.findall(regex, chapter['text'], re.U)) for regex in regexps])

    return characters


def parse_parts(roman):
    data = []

    for part in roman['parts']:
        for chapter in part['chapters']:
            data.append({
                'chapter': '{} - {}'.format(part['title'], chapter['title']),
                'rates': rate_characters(chapter)})

    return data

And huge code to convert native python data to plotly objects ready for plotting:

In [30]:
def prepare_data(data):
    data = data

    traces = []

    for character in reversed(sorted(data[0]['rates'])):
        traces.append(go.Scatter(
            x=[],
            y=[],
            text=[],
            fill='tonexty',
            mode='none',
            line={'shape': 'spline'},
            hoverinfo='text',
            name=character))

    sums = {}
    for trace in traces:
        for chapter in data:
            sums[chapter['chapter']] = sums.get(chapter['chapter'], 0) + chapter['rates'][trace['name']]
            trace['x'].append(chapter['chapter'])
            trace['y'].append(sums[chapter['chapter']])
            trace['text'].append(
                '{} - {}'.format(chapter['rates'][trace['name']], trace['name'])
                if chapter['rates'][trace['name']] else '')

    traces_with_liners = []
    for trace in traces:
        traces_with_liners.append(trace)
        traces_with_liners.append(go.Scatter(
            x=trace['x'],
            y=trace['y'],
            fill='tonexty',
            showlegend=False,
            line={'shape': 'spline'},
            hoverinfo='none',
            mode='none',
            fillcolor='#ffffff'
        ))

    return traces_with_liners


And returning to code from which we started — let's see the plot result:

In [31]:
roman = idiot.from_json()

data = parse_parts(roman)
traces = prepare_data(data)
plot(traces)